# LLM Inference with Burn on Google Colab

このノートブックは、Burn MLフレームワークを使用してGoogle Colab上でLLM推論を実行します。

**対応GPU:**
- Tesla T4 (無料版)
- A100 (Colab Pro+)
- H100 (Colab Enterprise)

**必要な設定:**
- ランタイムタイプ: GPU
- GPU種類: T4, A100, またはH100

## 1. GPU確認

In [ ]:
!nvidia-smi

## 2. Rustのインストール

In [ ]:
# Rustのインストール
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

# 環境変数の設定
import os
os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

# インストール確認
!rustc --version
!cargo --version

## 3. CUDA開発環境の確認

In [ ]:
# CUDAバージョン確認
!nvcc --version

# CUDA環境変数の設定
!echo $CUDA_HOME
!echo $LD_LIBRARY_PATH

## 4. プロジェクトのクローン/アップロード

In [ ]:
# 方法1: GitHubからクローン (リポジトリがある場合)
# !git clone https://github.com/YOUR_USERNAME/LLM-API.git
# %cd LLM-API

# 方法2: ローカルファイルをアップロード
# 1. 左サイドバーのファイルアイコンをクリック
# 2. LLM-APIフォルダ全体をドラッグ&ドロップ
# 3. 以下のセルを実行してディレクトリに移動

%cd /content/LLM-API

## 5. 依存関係のビルド

In [ ]:
# Cargo.tomlの確認
!cat Cargo.toml

In [ ]:
# ビルド (初回は10-20分かかります)
!cargo build --release

## 6. 推論の実行

In [ ]:
# 推論実行
!cargo run --release

## 7. カスタムプロンプトで推論

In [ ]:
# カスタムプロンプトを使用する場合は、main.rsを編集してください
# または、CLIオプションを追加する実装が必要です

## トラブルシューティング

### CUDAが見つからない場合
```bash
export CUDA_HOME=/usr/local/cuda
export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
export PATH=/usr/local/cuda/bin:$PATH
```

### メモリ不足の場合
- モデルサイズを小さくする（`ModelConfig::tiny()`を使用）
- より大きなGPU（A100やH100）を使用

### ビルドエラーの場合
```bash
cargo clean
cargo build --release
```